In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import Sequence
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from keras.callbacks import *
from tqdm import tqdm
from sklearn import metrics
import pickle
from keras.engine.topology import Layer
from keras import backend as K
import gc
from tensorflow.keras.backend import batch_dot
import tensorflow as tf
import re

In [ ]:
path = '../input/'
max_features = 120000
max_len = 70
max_word_len = 30
embed_size = 300
n_split = 5
n_epoch = 5

train_df = pd.read_csv(os.path.join(path, 'train.csv'))
test_df = pd.read_csv(os.path.join(path, 'test.csv'))

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, '{puncts}')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [ ]:
train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())

# Clean the text
train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
    
# Clean numbers
train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_numbers(x))
    
# Clean speelings
train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))



In [ ]:
##################################
##Prepare data for train/test
##################################

tr_X = train_df["question_text"].fillna("_na_").values
tr_y = train_df['target'].values
te_X = test_df["question_text"].fillna("_na_").values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(tr_X))
word_index = tokenizer.index_word
        
tr_X = tokenizer.texts_to_sequences(tr_X)
tr_X = pad_sequences(tr_X, maxlen=max_len, padding='post')       
te_X = tokenizer.texts_to_sequences(te_X)
te_X = pad_sequences(te_X, maxlen=max_len, padding='post')

In [ ]:
##################################
##create input function for train/test
##################################

#class input_creator(Sequence):
    
    #def __init__(self, x, y=None, batch_size=3):
        #super(input_creator, self).__init__()
        #self.x = x
        #self.y = y
        #self.batch_size = batch_size
        
    #def __getitem__(self, index):
        #start = self.batch_size * index
        #end = min(start + self.batch_size, len(self.x))
        #size  = end - start
        #batch_x = self.x[start:end, :]

        #if self.y is not None:
            #batch_y = self.y[start:end]
            #return batch_x, batch_y
        #else:
            #return batch_x

    #def __len__(self):
        #return (self.x.shape[0] + self.batch_size - 1) // self.batch_size

def input_creator(x, y, batch_size):
    while True: 
        for i in range((x.shape[0]//batch_size)+1):
            start = batch_size * i
            end = min(start+batch_size, x.shape[0])
            
            batch_x = x[start:end, :]
            batch_y = y[start:end]
            
            yield batch_x, batch_y

In [ ]:
## FUNCTIONS TAKEN FROM https://www.kaggle.com/gmhost/gru-capsule

def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    # Why random embedding for OOV? what if use mean?
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    #embedding_matrix = np.random.normal(emb_mean, 0, (nb_words, embed_size)) # std 0
    for i, word in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    #embedding_matrix = np.random.normal(emb_mean, 0, (nb_words, embed_size))
    for i, word in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for i, word in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

glove_embeddings = load_glove(word_index)
paragram_embeddings = load_para(word_index)
fasttext_embeddings = load_fasttext(word_index)

embedding_matrix = np.mean([glove_embeddings, paragram_embeddings, fasttext_embeddings], axis=0)

# vocab = build_vocab(df['question_text'])
# add_lower(embedding_matrix, vocab)
del glove_embeddings, paragram_embeddings, fasttext_embeddings
gc.collect()

np.shape(embedding_matrix)

In [ ]:
splits = list(StratifiedKFold(n_splits=n_split, shuffle=True, random_state=2018).split(tr_X, tr_y))

In [ ]:
class PositionEmbedding(Layer):

    def __init__(self, input_dim, output_dim,
                 mode='Add', **kwargs):

        self.input_dim   = input_dim #max_position_embeddings
        self.output_dim  = output_dim
        self.mode = mode
        super(PositionEmbedding, self).__init__(**kwargs)

    def build(self, input_shape):

        self.embeddings = self.add_weight(name='position_matrix',
                                 shape=(self.input_dim, self.output_dim),
                                 initializer='glorot_uniform',
                                         )
        super(PositionEmbedding, self).build(input_shape)

    def call(self, x):

        input_shape = K.shape(x)

        if self.mode == 'Add':
            batch_size, seq_len, output_dim = input_shape[0], input_shape[1], input_shape[2]
        else:
            batch_size, seq_len, output_dim = input_shape[0], input_shape[1], self.output_dim #concat

        #assert seq_len == self.input_dim
        position_embed = K.tile(K.expand_dims(self.embeddings[:seq_len, :self.output_dim], axis=0), K.stack([batch_size, 1, 1]),)
        
        if self.mode == 'Add':
            return x + position_embed
        
        return K.concatenate([x, position_embed], axis=-1)
    def compute_output_shape(self, input_shape):
        if self.mode == 'Concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.output_dim)
        return input_shape


In [ ]:
class ConvSelfAttention(Layer):

    def __init__(self, kernel_size, n_head, output_dim, use_bias=True, V_activation='elu', **kwargs):
        
        self.kernel_size = kernel_size
        self.output_dim = output_dim
        self.n_head = n_head
        self.use_bias = use_bias
        self.V_activation = V_activation
        self.per_head_dim = int(self.output_dim / self.n_head)
        super(ConvSelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):

        self.WQ = self.add_weight(name='WQ',
                                  shape=(self.kernel_size, input_shape[-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(self.kernel_size, input_shape[-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(self.kernel_size, input_shape[-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)

        if self.use_bias:
            self.BQ = self.add_weight(name='BQ',
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)
            self.BK = self.add_weight(name='BK',
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)
            self.BV = self.add_weight(name='BV',
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)

        super(ConvSelfAttention, self).build(input_shape)

    def call(self, x):

        Q_seq = x #Query Embeddings
        K_seq = x #Key Embeddings
        V_seq = x #Value Embeddings

        Q_seq = self.feature_extraction(Q_seq, self.WQ, self.BQ, activation='elu')
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.n_head, self.per_head_dim)) #(batch_size, max_len-kernel_size + 1, n_head, per_head_dim)
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = self.feature_extraction(K_seq, self.WK, self.BK, activation='elu') #(batch_size, max_len-kernel_size + 1, n_head, per_head_dim)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.n_head, self.per_head_dim))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = self.feature_extraction(V_seq, self.WV, self.BV, activation=self.V_activation) #(batch_size, max_len-kernel_size + 1, n_head, per_head_dim)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.n_head, self.per_head_dim))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))#(batch_size, n_head, max_len-kernel_size + 1, per_head_dim)

        A = batch_dot(Q_seq, K_seq, axes=[3,3]) / self.per_head_dim**0.5 #(batch_size, n_head, max_len-kernel_size + 1, max_len-kernel_size+1)
        A = K.softmax(A) #(batch_size n_head, max_len-kernel_size + 1, max_len-kernel_size+1)
        A = batch_dot(A, V_seq, axes=[3,2]) 
        A = K.permute_dimensions(A, (0,2,1,3))
        A = K.reshape(A, (-1, K.shape(A)[1], self.output_dim))

        return A

    def feature_extraction(self, inp, kernel, bias, activation='elu'):

        x = K.conv1d(inp, kernel=kernel, padding='same', strides=1)
        if self.use_bias:
            x = K.bias_add(x, bias)
        if activation == 'elu':
            x = K.elu(x)
        elif activation == 'tanh':
            x = K.tanh(x)
        else:
            x = K.relu(x)

        return x

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.output_dim)

In [ ]:
class LayerNormalization(Layer):

    def __init__(self, eps=1e-6, **kwargs):

        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.gamma  = self.add_weight(name='gamma',
                                     shape=(input_shape[-1:]),
                                     initializer='glorot_uniform',
                                     trainable=True)
        self.beta = self.add_weight(name='bata',
                                  shape=(input_shape[-1:]),
                                  initializer='glorot_uniform',
                                  trainable=True)

        super(LayerNormalization,self).build(input_shape)

    def call(self, x):
        
        means = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        x_norm = (x - means) / ((std + self.eps)**0.5)
        output = self.gamma * x_norm + self.beta

        return output

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
class ConvTargetAttention(Layer):

    def __init__(self, kernel_size, n_head, output_dim, use_bias=True, **kwargs):

        self.kernel_size = kernel_size
        self.output_dim = output_dim
        self.n_head = n_head
        self.use_bias = use_bias
        self.per_head_dim = int(self.output_dim / self.n_head)
        super(ConvTargetAttention, self).__init__(**kwargs)

    def build(self, input_shape):

        self.WK = self.add_weight(name='WK',
                                  shape=(self.kernel_size, input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(self.kernel_size, input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)

        if self.use_bias:
            self.BK = self.add_weight(name='BK',
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)
            self.BV = self.add_weight(name='BV',
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)

        super(ConvTargetAttention, self).build(input_shape)

    def call(self, inputs):

        assert len(inputs) == 3
        T_seq, K_seq, V_seq = inputs # T:(batch_size, embed_size)
        T_seq = K.reshape(T_seq, (-1, self.n_head, self.per_head_dim))

        K_seq = self.feature_extraction(K_seq, self.WK, self.BK, activation='elu')
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.n_head, self.per_head_dim))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))  #(batch_size, n_head, seq_len-kernel_size+1, self.per_head_dim)
        V_seq = self.feature_extraction(V_seq, self.WV, self.BV, activation='elu')
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.n_head, self.per_head_dim))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3)) #(batch_size, n_head, seq_len-kernel_size+1, self.per_head_dim)

        A = batch_dot(T_seq, K_seq, axes=[2, 3]) / self.per_head_dim**0.5
        A = K.softmax(A)
        A = batch_dot(A, V_seq, axes=[1, 2])
        A = K.reshape(A, (-1, self.output_dim))

        return A

    def feature_extraction(self, inp, kernel, bias, activation='elu'):

            x = K.conv1d(inp, kernel=kernel)
            if self.use_bias:
                x = K.bias_add(x, bias)
            if activation == 'elu':
                x = K.elu(x)
            elif activation == 'tanh':
                x = K.tanh(x)

            return x

    def compute_output_shape(self, input_shape):
        return input_shape[0][0], input_shape[0][1]

In [ ]:
#The code from 
#https://www.kaggle.com/braquino/5-fold-lstm-attention-fully-commented-0-694

def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [ ]:
class main(object):

    def __init__(self, max_features, max_len, pretrain_embed_matrix=None, embed_size=300):
        
        self.max_features = max_features
        self.max_len = max_len
        self.output_dim = 60
        self.pretrain_embed_matrix = pretrain_embed_matrix
        self.embed_size = embed_size

    def build(self):

        Inp = Input(shape=(self.max_len,))
        
        x  = Embedding(self.max_features, self.embed_size, weights=[self.pretrain_embed_matrix])(Inp)
        x  = PositionEmbedding(self.max_len, self.embed_size)(x)
        #x  = Dropout(0.1)(x)
        
        a_1 = ConvSelfAttention(kernel_size=3, n_head=6, output_dim=self.embed_size)(x)
        a_2 = ConvSelfAttention(kernel_size=3, n_head=6, output_dim=self.embed_size, V_activation='tanh')(x)
        a   = Lambda(lambda x: x[0] * x[1])([a_1, a_2])
        M   = Add()([x, a]) # The Idea From Attention All you Need
        M   = LayerNormalization()(M)
        
        #applied to each position separately and identically. 
        #representing a token of the input, 
        #and the same process is repeated for as many tokens as there are in the sentence.
        F   = Conv1D(self.embed_size*2, kernel_size=1, activation='relu')(M)
        F   = Conv1D(self.embed_size, kernel_size=1)(F)
        F   = Add()([F, M])
        F   = LayerNormalization()(F)
        
        x   = GlobalAvgPool1D()(F)
        #x   = BatchNormalization()(x)
        x   = Dropout(0.1)(x)
        x   = Dense(1, activation='sigmoid', use_bias=True)(x)
        
        model = Model(Inp, x) 

        return model
    
model = main(max_features=max_features, max_len=max_len, pretrain_embed_matrix=embedding_matrix).build()
model.summary()

In [ ]:
#https://arxiv.org/pdf/1608.03983.pdf
#/1506.01186.pdf

class cyclr(Callback):

    def __init__(self, max_lr, base_lr, step_size, gamma=None, mode='triangular'):

        self.max_lr =  max_lr
        self.base_lr = base_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma

        self.clr_iteration = 0
        self.trn_iteration = 0

        if self.mode == 'triangular':
            self.scale_fn = lambda x:1
        elif self.mode == 'triangular2':
            self.scale_fn = lambda x: 1/(2.**(x-1))
        elif self.mode == 'exp_range':
            self.scale_fn = lambda x: self.gamma**(x)

        self.history = {}
        self._reset()
    
    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):

        cycle = np.floor(1 + self.clr_iteration/(2*self.step_size))
        x = np.abs(self.clr_iteration/self.step_size-2*cycle)
        if self.mode == 'exp_range':
            lr = self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1-x))*self.scale_fn(self.clr_iteration)
        else:
            lr = self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1-x))*self.scale_fn(cycle)

        return lr
    
    def on_train_begin(self, logs={}):
        logs = logs or {}
        
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs={}):
        logs = logs or {}
        self.trn_iteration +=1
        self.clr_iteration +=1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
cyc_lr = cyclr(max_lr=0.003, base_lr=0.001, step_size=300, gamma=0.99994, mode='exp_range')

model.compile(optimizer=Adam(), loss=['binary_crossentropy'], metrics=[matthews_correlation, 'binary_accuracy'])
callbacks=[cyc_lr,
           #EarlyStopping(monitor='val_loss', patience=1, min_delta=0.0001, verbose=1),
           ModelCheckpoint('my_model.h5', save_best_only=True, save_weights_only=True,
                          verbose=1, monitor='val_matthews_correlation', mode='max')]

In [ ]:
for i, split in enumerate(splits):
    x_train_fold, y_train_fold, x_val_fold, y_val_fold = tr_X[split[0]],tr_y[split[0]],tr_X[split[1]],tr_y[split[1]]
    
    print('fold {}'.format(i))
    print('Start Train')
    model.fit_generator(input_creator(x_train_fold, y_train_fold, 512), steps_per_epoch= (x_train_fold.shape[0]//512)+1, 
                        epochs=5, validation_data=(x_val_fold, y_val_fold),shuffle=True, 
                        callbacks=callbacks, )
    print('End Train')    


In [ ]:
del model 
model2 = main(max_features=max_features, max_len=max_len, pretrain_embed_matrix=embedding_matrix).build()
model2.load_weights('my_model.h5')

best_thresh = 0.1
best_score = 0.0

tr_pred = model2.predict(tr_X, batch_size=800, verbose=1)

for thresh in [i * 0.01 for i in range(100)]:
    thresh = np.round(thresh, 2)
    score = metrics.f1_score(tr_y, (tr_pred < thresh).astype(int))
    if score > best_score:
        best_score = score
        best_thresh = thresh
    print("F1 score at threshold {0} is {1}".format(thresh, score))
print(best_thresh)

In [ ]:
for thresh in [i * 0.01 for i in range(100)]:
    thresh = np.round(thresh, 2)
    score = metrics.f1_score(tr_y, (tr_pred < thresh).astype(int))
    if score > best_score:
        best_score = score
        best_thresh = thresh
    print("F1 score at threshold {0} is {1}".format(thresh, score))
print(best_thresh)

In [ ]:
output = model2.predict(te_X, batch_size=800, verbose=1)
output = (output < best_thresh).astype(int)
output = output.flatten()

submission = pd.DataFrame({'qid':test_df.qid.values,
                          'prediction':output})

print(submission.head())
submission.to_csv('submission.csv', index=False)
print(os.listdir('./'))